In [15]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
import json

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage

# Initialize two LLMs
llm1 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)
llm2 = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.7)

In [21]:
# Initialize conversation log
conversation_log = []

# Set persona for each LLM in system messages
system_message1 = SystemMessage(content="""대화하는 사람의 말에 반박하되:
1. 이전 대화 내용을 그대로 반복하지 마세요
2. 새로운 관점이나 논리를 제시하세요
3. 구체적인 예시를 들어 설명하세요""")
system_message2 = SystemMessage(content="""대화하는 사람의 말에 반박하되:
1. 이전과 다른 새로운 논리로 반박하세요
2. 실제 사례나 연구 결과를 인용해보세요
3. 상대방 주장의 허점을 지적하세요""")

# Set an initial prompt for the first LLM
initial_prompt = "안녕"

# Initialize game state variables
total_round = 1
answer_count = 0
max_rounds = 10  # Maximum number of rounds allowed

# Start the conversation
print("Starting conversation between LLM1 and LLM2 with personas set in system messages...\n")
response1 = llm1([system_message1, HumanMessage(content=initial_prompt)])
conversation_log.append({"LLM1": initial_prompt})
conversation_log.append({"LLM2": response1.content})
print(f"LLM1: {initial_prompt}")
print(f"LLM2: {response1.content}\n")

# Iterate conversation for a few rounds
for round_num in range(max_rounds):
    # Process LLM1's response and validate total_round
    prompt1 = response1.content
    response2 = llm2([system_message2, HumanMessage(content=prompt1)])
    conversation_log.append({"LLM1": prompt1})
    conversation_log.append({"LLM2": response2.content})
    print(f"LLM1: {prompt1}")
    print(f"LLM2: {response2.content}\n")

    prompt2 = response2.content
    response1 = llm1([system_message1, HumanMessage(content=prompt2)])
    conversation_log.append({"LLM2": prompt2})
    conversation_log.append({"LLM1": response1.content})
    print(f"LLM2: {prompt2}")
    print(f"LLM1: {response1.content}\n")

    # Parse LLM1 output (expected in JSON format) and validate game state
    try:
      if response1.content.startswith("{"):
        llm1_output = json.loads(response1.content)  # Parse JSON safely
        if llm1_output.get("total_round", total_round) != total_round:
            print(f"Correcting total_round from {llm1_output['total_round']} to {total_round}")
            llm1_output["total_round"] = total_round
        if llm1_output.get("check_answer", False):
            answer_count += 1
        total_round += 1
        if total_round > max_rounds or llm1_output.get("is_end", False):
            break
      else:
        llm1_output = {"message": response1.content}
    except json.JSONDecodeError as e:
        print(f"Error parsing LLM1 output: {e}")
        break

# Save conversation log to a file
with open("conversation_log.txt", "w", encoding="utf-8") as log_file:
    for entry in conversation_log:
        for key, value in entry.items():
            log_file.write(f"{key}: {value}\n")
    print("\nConversation log saved to 'conversation_log.txt'.")


Starting conversation between LLM1 and LLM2 with personas set in system messages...

LLM1: 안녕
LLM2: 안녕하세요, 하지만 인사하는 것만으로는 대화가 깊어지지 않으니 다른 주제로 이야기를 나눠보는 게 좋겠어요.

LLM1: 안녕하세요, 하지만 인사하는 것만으로는 대화가 깊어지지 않으니 다른 주제로 이야기를 나눠보는 게 좋겠어요.
LLM2: 안녕하세요! 하지만 인사하는 것도 대화의 중요한 시작점입니다. 다른 주제로 넘어가는 것이 꼭 더 깊은 대화를 보장하지는 않아요. 인사를 통해서도 충분히 서로의 기분이나 상황을 알 수 있으니까요.

LLM2: 안녕하세요! 하지만 인사하는 것도 대화의 중요한 시작점입니다. 다른 주제로 넘어가는 것이 꼭 더 깊은 대화를 보장하지는 않아요. 인사를 통해서도 충분히 서로의 기분이나 상황을 알 수 있으니까요.
LLM1: 안녕하세요! 하지만 인사가 대화의 시작점이라고 해도, 너무 오래 인사에만 집중하는 것은 오히려 대화를 지루하게 만들 수 있습니다. 대화의 깊이를 더하기 위해서는 다양한 주제로 자연스럽게 넘어가는 것이 더 효과적이라고 생각합니다. 인사는 기본적인 예의일 뿐, 대화의 본질은 그 이후의 내용에서 결정된다고 볼 수 있습니다.

LLM1: 안녕하세요! 하지만 인사가 대화의 시작점이라고 해도, 너무 오래 인사에만 집중하는 것은 오히려 대화를 지루하게 만들 수 있습니다. 대화의 깊이를 더하기 위해서는 다양한 주제로 자연스럽게 넘어가는 것이 더 효과적이라고 생각합니다. 인사는 기본적인 예의일 뿐, 대화의 본질은 그 이후의 내용에서 결정된다고 볼 수 있습니다.
LLM2: 안녕하세요! 물론 인사가 대화의 시작점일 수 있지만, 인사의 중요성을 간과해서는 안 됩니다. 인사는 사람들 간의 관계를 형성하고, 서로의 감정을 존중하는 중요한 역할을 합니다. 인사를 소홀히 하면 대화의 분위기가 어색해질 수 있으며, 대화의 깊이를 더하기 전에 기본적인 예의가 지켜져야 합니다. 